Cell 1: Install dependencies

In [ ]:
!pip install -q transformers accelerate peft datasets trl bitsandbytes

Cell 2: Import packages

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import load_dataset
import torch


Cell 3: Load Mistral-7B base model + tokenizer

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)


Cell 4: Apply LoRA adapter

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


Cell 5: Load your dataset (generated)

In [ ]:
# Upload your auto-generated file here
dataset = load_dataset("json", data_files="fin_llm_instruct.jsonl")


Cell 6: Format for Chat-style tuning

In [ ]:
def format_instruction(example):
    return {
        "input_ids": tokenizer(
            f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}",
            truncation=True, padding="max_length", max_length=512, return_tensors="pt"
        )["input_ids"].squeeze()
    }

tokenized_dataset = dataset["train"].map(format_instruction)


Cell 7: Set training args + start training

In [ ]:
training_args = TrainingArguments(
    output_dir="fin-llm-mistral",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_dir="./logs",
    save_strategy="epoch"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=512
)

trainer.train()


Cell 8: Save fine-tuned model

In [ ]:
model.save_pretrained("fin_llm_mistral_lora")
tokenizer.save_pretrained("fin_llm_mistral_lora")
